## Embeddings

Embeddings are integral to various natural language processing applications, with their quality crucial for optimal performance. They are commonly used in knowledge bases to represent textual data as dense vectors enabling efficient similarity search and retrieval. In Retrieval Augmented Generation (RAG), embeddings are used to retrieve relevant passages from a corpus to provide context for language models to generate informed, knowledge-grounded responses. Embeddings also play a key role in personalization and recommendation systems by representing user preferences, item characteristics, and historical interactions as vectors, allowing calculation of similarities for personalized recommendations based on user behavior and item embeddings. As new embedding models are released with incremental quality improvements, organizations must weigh the potential benefits against the associated costs of upgrading, considering factors like computational resources, data preprocessing, integration efforts, and projected performance gains impacting business metrics.

#### How a piece of text is converted into a vector?
Common approach is to use models which can provide contextualized embeddings for entire sentences. These models are based on deep learning architectures such as Transformers, which can capture the contextual information and relationships between words in a sentence more effectively.

![Embedding Model](./images/vector_embedding.png)

In addition to semantic search, you can use embeddings to augment your prompts for more accurate results through Retrieval Augmented Generation (RAG)—but in order to use them, you’ll need to store them in a database with vector capabilities.

![Embedding Model](./images/vector_db.jpg)

In [ ]:
#%pip install langchain_cohere -q
#%pip install spacy -q
#%pip install python-dotenv -q
#ignore error

In [ ]:
# now you need to run this in a terminal window
# python -m spacy download en_core_web_md
# now restart your kernel

Standard imports for the libraires we will be using in this notebook.  Try to keep your imports in the first cell so this can this code can more easliy be converted into a python program later

In [37]:
import boto3
import pandas as pd
import json
import time
import os
import numpy as np
import pyarrow
import traceback
from langchain.embeddings import BedrockEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
from langchain.embeddings import SpacyEmbeddings
from dotenv import load_dotenv

load_dotenv()


True

#### Boto3 client to connect to the model

In [18]:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(assumed_role: Optional[str] = None, region: Optional[str] = 'us-west-2',runtime: Optional[bool] = True,external_id=None, ep_url=None):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides 
    """
    target_region = region

    print(f"Create new client\n  Using region: {target_region}:external_id={external_id}: ")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    # 🔍 Add this right after session is created
    creds = session.get_credentials().get_frozen_credentials()
    print(f"✅ Using Access Key: {creds.access_key[:4]}...{creds.access_key[-4:]}")

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        if external_id:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
                ExternalId=external_id
            )
        else:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
            )
        print(f"Using role: {assumed_role} ... sts::successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    if ep_url:
        bedrock_client = session.client(service_name=service_name,config=retry_config,endpoint_url = ep_url, **client_kwargs )
    else:
        bedrock_client = session.client(service_name=service_name,config=retry_config, **client_kwargs )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client
    

#### Helper Class to connect and run the embeddings

In [19]:
import json
import boto3

class TitanEmbeddings(object):
    accept = "application/json"
    content_type = "application/json"
    
    def __init__(self, model_id="amazon.titan-embed-text-v2:0", boto3_client=None, region_name='us-west-2'):
        
        if boto3_client:
            self.bedrock_boto3 = boto3_client
        else:
            # self.bedrock_boto3 = boto3.client(service_name='bedrock-runtime')
            self.bedrock_boto3 = boto3.client(
                service_name='bedrock-runtime', 
                region_name=region_name, 
            )
        self.model_id = model_id

    def __call__(self, text, dimensions, normalize=True):
        """
        Returns Titan Embeddings

        Args:
            text (str): text to embed
            dimensions (int): Number of output dimensions.
            normalize (bool): Whether to return the normalized embedding or not.

        Return:
            List[float]: Embedding
            
        """

        body = json.dumps({
            "inputText": text,
            "dimensions": dimensions,
            "normalize": normalize
        })

        response = self.bedrock_boto3.invoke_model(
            body=body, modelId=self.model_id, accept=self.accept, contentType=self.content_type
        )

        response_body = json.loads(response.get('body').read())

        return response_body['embedding']


In [20]:
import json
import os
import sys

import boto3

aws_client = get_bedrock_client() #boto3.client('bedrock')

bedrock_embeddings = TitanEmbeddings(model_id="amazon.titan-embed-text-v2:0", boto3_client=aws_client)
bedrock_embeddings
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

Create new client
  Using region: us-west-2:external_id=None: 
✅ Using Access Key: AKIA...Q65M
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


### Generate Embeddings

At the time of writing you can use amazon.titan-embed-text-v2 as embedding model via the API. The input text size is 8k tokens and the output vector length can be any of 256, 512 or 1024

To use a text embeddings model, use the InvokeModel API operation or the Python SDK. Use InvokeModel to retrieve the vector representation of the input text from the specified model.
Input

```

{
    "inputText": text,
    "dimensions": dimensions, # range from 256 , 512, 1024
    "normalize": normalize
}

Output

{
    "embedding": []
}
```

#### Normalization of a vector 

Normalization is the process of scaling it to have a unit length or magnitude of 1. It is useful to ensure that all vectors have the same scale and contribute equally during vector operations, preventing some vectors from dominating others due to their larger magnitudes.

#### When should you Normalize:
Use this as default for most of the use cases like Retrieval, RAG and others

#### When you should not Normalize: 
Normally normalization wil work for all use cases, but you can experiment for certain use cases like Classification or Entity extraction


In [21]:
prompt_data = "Amazon Bedrock supports foundation models from industry-leading providers such as \
AI21 Labs, Anthropic, Stability AI, and Amazon. Choose the model that is best suited to achieving \
your unique goals."


modelId = "amazon.titan-embed-text-v2:0"  # 
accept = "application/json"
contentType = "application/json"



sample_model_input={
    "inputText": prompt_data,
    "dimensions": 256,
    "normalize": True
}

body = json.dumps(sample_model_input)

response = aws_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

embedding = response_body.get("embedding")
print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

The embedding vector has 256 values
[-0.1435401737689972, 0.06038207560777664, -0.05212177708745003, '...', 0.15111945569515228, 0.05338708311319351, 0.002391696209087968]


#### Lets define functions that will use various embedding models so we can generate vector embeddings
Spacey

In [22]:
def generate_spacy_vector_embedding(text):
    embedder = SpacyEmbeddings(model_name="en_core_web_md")
    query_embedding = embedder.embed_query(text)

    return(np.array(query_embedding))

Cohere

In [23]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': [text_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


Amazon Titan

In [24]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



In [25]:
# Let's generate a dense vector using Amazon Titan without using a np.array as a return value
def generate_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    #Note pgvector does not want a np.array as out manual method
    return(embedding)



This is the mathmatical formula to calcuate cosine similarity between 2 vectors

In [26]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



In [27]:
def clean_value(value):
    value_str = str(value)
    cleaned_value = ''.join(char for char in value_str if char.isalnum() or char.isspace())
    return cleaned_value

In [28]:
def limit_string_size(x, max_chars=2048):
    # Check if the input is a string
    if isinstance(x, str):
        return x[:max_chars]
    else:
        return x

In [29]:
def print_top_values(list_stuff: list, num_items: int) -> None:
    i=0
    for item in list_stuff:
        i=i+1
        if i>num_items:
            return None
        print(item)

In [30]:
#Titan
king_vector = generate_titan_vector_embedding("King")
queen_vector = generate_titan_vector_embedding("Queen")
man_vector = generate_titan_vector_embedding("man")
woman_vector = generate_titan_vector_embedding("woman")
print(f"This embedding has {len(king_vector)} dimensions")
print(king_vector[:5])

/var/folders/m1/tx6ck0f962jd1hq4ch5phg_80000gn/T/ipykernel_36750/1231035754.py:4: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  embeddings_client = BedrockEmbeddings(region_name="us-west-2")


This embedding has 1536 dimensions
[-0.25       -0.91015625  0.55078125  0.59375     0.5546875 ]


In [31]:
calculated_queen_vector = king_vector - man_vector + woman_vector

similarity = cosine_similarity(calculated_queen_vector, queen_vector)
print(f"Cosine Similarity distance between Titan calculated_queen - Queen: {similarity:.4f}")

similarity = cosine_similarity(king_vector, queen_vector)
print(f"Cosine Similarity of Titan King to Queen: {similarity:.4f}")

Cosine Similarity distance between Titan calculated_queen - Queen: 0.6676
Cosine Similarity of Titan King to Queen: 0.6291


In [32]:
# Input cohere for embedding 
king_vector = generate_cohere_vector_embedding('King')
queen_vector = generate_cohere_vector_embedding("Queen")
man_vector = generate_cohere_vector_embedding("man")
woman_vector = generate_cohere_vector_embedding("woman")
print(f"This embedding has {len(king_vector)} dimensions")
print(king_vector[:5])

This embedding has 1024 dimensions
[ 0.02383423 -0.0247345   0.02523804 -0.03985596 -0.06323242]


In [33]:
calculated_queen_vector = king_vector - man_vector + woman_vector

similarity = cosine_similarity(calculated_queen_vector, queen_vector)
print(f"Cosine Similarity distance between Cohere calculated_queen - Queen: {similarity:.4f}")

similarity = cosine_similarity(king_vector, queen_vector)
print(f"Cosine Similarity of Cohere King to Queen: {similarity:.4f}")

Cosine Similarity distance between Cohere calculated_queen - Queen: 0.7225
Cosine Similarity of Cohere King to Queen: 0.7343


In [39]:
#Spacy
king_vector = generate_spacy_vector_embedding("King")
queen_vector = generate_spacy_vector_embedding("Queen")
man_vector = generate_spacy_vector_embedding("man")
woman_vector = generate_spacy_vector_embedding("woman")
print(f"This embedding has {len(king_vector)} dimensions")
print(king_vector[:5])

This embedding has 300 dimensions
[-0.60644001 -0.51204997  0.0064921  -0.29194    -0.56515002]


In [40]:
calculated_queen_vector = king_vector - man_vector + woman_vector

similarity = cosine_similarity(calculated_queen_vector, queen_vector)
print(f"Cosine Similarity distance between Spacey calculated_queen - Queen: {similarity:.4f}")

similarity = cosine_similarity(king_vector, queen_vector)
print(f"Cosine Similarity of Spacey King to Queen: {similarity:.4f}")

Cosine Similarity distance between Spacey calculated_queen - Queen: 0.4812
Cosine Similarity of Spacey King to Queen: 0.3825


Let's examine other phrases

In [41]:
similarity = cosine_similarity(generate_titan_vector_embedding("cat"), generate_titan_vector_embedding("book"))
print(f"Cosine Similarity of cat to book using Titan: {similarity:.4f}")

Cosine Similarity of cat to book using Titan: 0.3514


In [42]:
similarity = cosine_similarity(generate_cohere_vector_embedding("cat"), generate_cohere_vector_embedding("book"))
print(f"Cosine Similarity of cat to book using Cohere: {similarity:.4f}")

Cosine Similarity of cat to book using Cohere: 0.5562


In [43]:
similarity = cosine_similarity(generate_spacy_vector_embedding("cat"), generate_spacy_vector_embedding("book"))
print(f"Cosine Similarity of cat to book using Spacey: {similarity:.4f}")

Cosine Similarity of cat to book using Spacey: 0.0693


Now let's look at a larger sentences and see how larger models with more complexity handle the same task Here are 2 sentences that semantically similar but use different words and phrasing.

The majestic, towering skyscrapers, their gleaming windows reflecting the golden rays of the setting sun, stood as a testament to human ingenuity and the indomitable spirit of progress, while the bustling streets below teemed with life as people from all walks of life hurried to their destinations, their faces a mix of determination and weariness, yet each individual contributing to the vibrant tapestry of the city's existence.

The awe-inspiring, colossal high-rises, their polished glass facades mirroring the warm, amber glow of the fading daylight, served as a powerful symbol of human innovation and the unyielding drive for advancement, as the lively thoroughfares beneath pulsed with energy, filled with individuals from diverse backgrounds rushing to their intended locations, their expressions an amalgamation of resolve and fatigue, yet all playing a vital role in the dynamic, intricate mosaic that shaped the city's vibrant identity.

In [44]:
sentence1 = "The majestic, towering skyscrapers, their gleaming windows reflecting the golden rays of the setting sun, stood as a testament to human ingenuity and the indomitable spirit of progress, while the bustling streets below teemed with life as people from all walks of life hurried to their destinations, their faces a mix of determination and weariness, yet each individual contributing to the vibrant tapestry of the city's existence."
sentence2 = "The awe-inspiring, colossal high-rises, their polished glass facades mirroring the warm, amber glow of the fading daylight, served as a powerful symbol of human innovation and the unyielding drive for advancement, as the lively thoroughfares beneath pulsed with energy, filled with individuals from diverse backgrounds rushing to their intended locations, their expressions an amalgamation of resolve and fatigue, yet all playing a vital role in the dynamic, intricate mosaic that shaped the city's vibrant identity."
similarity = cosine_similarity(generate_spacy_vector_embedding(sentence1), generate_spacy_vector_embedding(sentence2))
print(f"Cosine Similarity of S1 to S2 using Spacey: {similarity:.4f}")

Cosine Similarity of S1 to S2 using Spacey: 0.9637


In [45]:
sentence1 = "The majestic, towering skyscrapers, their gleaming windows reflecting the golden rays of the setting sun, stood as a testament to human ingenuity and the indomitable spirit of progress, while the bustling streets below teemed with life as people from all walks of life hurried to their destinations, their faces a mix of determination and weariness, yet each individual contributing to the vibrant tapestry of the city's existence."
sentence2 = "The awe-inspiring, colossal high-rises, their polished glass facades mirroring the warm, amber glow of the fading daylight, served as a powerful symbol of human innovation and the unyielding drive for advancement, as the lively thoroughfares beneath pulsed with energy, filled with individuals from diverse backgrounds rushing to their intended locations, their expressions an amalgamation of resolve and fatigue, yet all playing a vital role in the dynamic, intricate mosaic that shaped the city's vibrant identity."
similarity = cosine_similarity(generate_titan_vector_embedding(sentence1), generate_titan_vector_embedding(sentence2))
print(f"Cosine Similarity of S1 to S2 using Titan: {similarity:.4f}")

Cosine Similarity of S1 to S2 using Titan: 0.8726


In [46]:
sentence1 = "The majestic, towering skyscrapers, their gleaming windows reflecting the golden rays of the setting sun, stood as a testament to human ingenuity and the indomitable spirit of progress, while the bustling streets below teemed with life as people from all walks of life hurried to their destinations, their faces a mix of determination and weariness, yet each individual contributing to the vibrant tapestry of the city's existence."
sentence2 = "The awe-inspiring, colossal high-rises, their polished glass facades mirroring the warm, amber glow of the fading daylight, served as a powerful symbol of human innovation and the unyielding drive for advancement, as the lively thoroughfares beneath pulsed with energy, filled with individuals from diverse backgrounds rushing to their intended locations, their expressions an amalgamation of resolve and fatigue, yet all playing a vital role in the dynamic, intricate mosaic that shaped the city's vibrant identity."
similarity = cosine_similarity(generate_cohere_vector_embedding(sentence1), generate_cohere_vector_embedding(sentence2))
print(f"Cosine Similarity of S1 to S2 using Cohere: {similarity:.4f}")

Cosine Similarity of S1 to S2 using Cohere: 0.8160
